In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

from siuba import *
import pandas as pd
import geopandas as gpd
import datetime as dt

from rt_analysis import rt_parser
from rt_analysis import rt_filter_map_plot

import shared_utils

In [2]:
from rt_analysis import sccp_tools
import numpy as np

In [3]:
from tqdm.notebook import tqdm

## run Beaumont (2024, legacy pipeline)

In [41]:
dates = shared_utils.rt_dates.oct2024_week

In [16]:
dates = [dt.date.fromisoformat(date) for date in dates]

In [17]:
dates

[datetime.date(2024, 10, 14),
 datetime.date(2024, 10, 15),
 datetime.date(2024, 10, 16),
 datetime.date(2024, 10, 17),
 datetime.date(2024, 10, 18),
 datetime.date(2024, 10, 19),
 datetime.date(2024, 10, 20),
 datetime.date(2024, 10, 21)]

In [13]:
from tqdm.notebook import tqdm

In [14]:
pbar = tqdm()

0it [00:00, ?it/s]

In [21]:
# for analysis_date in dates:
#     print(f'{analysis_date}...', end=None)
#     try:
#         rt_day = rt_parser.OperatorDayAnalysis(34,
#                                                analysis_date, pbar)
#         rt_day.export_views_gcs()
#         print('done')
#     except:
#         print('failed')

#  ran OK except no Sunday service

In [22]:
beaumont = gpd.read_file('./corridors_sccp_lpp/2024/beaumont_map.geojson')

In [25]:
beaumont['id'] = range(9)

In [29]:
beaumont_a = beaumont >> filter(_.id.isin(range(5)))
beaumont_a = beaumont_a.dissolve()

In [30]:
beaumont_a.explore()

In [31]:
rt_day = rt_filter_map_plot.from_gcs(34, dates[0])

found shapes parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/shapes_34_2024-10-14.parquet


In [32]:
rt_day.add_corridor(beaumont_a)

In [34]:
rt_day.corridor

,geometry
0,"POLYGON ((274071.272 -450179.931, 274017.191 -..."


In [36]:
rt_day.corridor['distance_meters'] = 0

In [38]:
rt_day.corridor_metrics(sccp=True)

metrics attached to self.corridor: 


,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,schedule_delay_minutes,total_speed_delay,total_schedule_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,geometry
0,1381,NaN,City of Beaumont,51.5,57.2,62.7,-166.5,-0.9,-135.3,2.0,28.0,6,8.1,0.4,-5.9,0.0,0.0,0.0,16,"POLYGON ((274071.272 -450179.931, 274017.191 -..."
1,3574,NaN,City of Beaumont,11.5,14.3,14.8,31.2,2.9,-135.3,2.0,25.1,14,8.3,0.8,1.2,0.9,0.0,NaN,16,"POLYGON ((274071.272 -450179.931, 274017.191 -..."


In [39]:
rt_day.quick_map_corridor()

In [42]:
beau_corridor_metrics = sccp_tools.sccp_average_metrics(itp_id=34, date_range=dates, corridor=beaumont_a)

AttributeError: 'str' object has no attribute 'astype'

octa_corridor_metrics = sccp_tools.sccp_average_metrics(itp_id=235, date_range=date_range, corridor=octa)# About the Metrics

The schedule-based metric is a daily average of the sum of median trip stop delays along the corridor. To further explain, we take each corridor trip that we have data for and look at the delay in comparison to the schedule at each stop, after subtracting off any delay present as the trip entered the corridor. For each trip we then take the median delay of all stops along the corridor, and sum these medians to create the metric.

The speed-based metric is a daily average of the sum of delays for each trip traversing the corridor as compared to a reference speed of 16 miles per houar. To further explain, we take each corridor trip that we have data for and calculate the hypothetical time it would take for that trip to traverse the corridor at a speed of 16 mph. The difference between the actual time it took for the trip to traverse the corridor and that hypothetical time is the speed-based delay for that trip, and we sum those delays to create the metric. This metric is intended to provide a more consistent basis for comparison independent of scheduling practices.

In other words, if we expect a hypothetical bus lane/signal priority/payment system etc to increase corridor speeds to 16mph, this is how much time we could save per day.

With corridor attached, generate both metrics using `RtFilterMapper.corridor_metrics()`

## Common Date Range (2022-23) and convienience function

* both the date range and tools to generate data are likely to change in future cycles
* reference documentation in `rt_delay`

In [ ]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

# OCTA 12/1/2022

In [ ]:
octa = gpd.read_file('./corridors_sccp_lpp/octa_dec1_rev.geojson')

In [ ]:
# octa.explore()

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(235, dt.date(2022, 5, 5))

In [ ]:
rt_day.add_corridor(octa)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
octa_corridor_metrics = sccp_tools.sccp_average_metrics(itp_id=235, date_range=date_range, corridor=octa)

In [ ]:
octa_corridor_metrics

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
rt_day.segment_speed_map(corridor=True)

In [ ]:
rt_day.stop_delay_view >> filter(_.corridor) >> distinct(_.route_short_name)

# AC Dec 15

* applicant also requested BART, unable without vehicle positions and as a grade-seperated system possibly non-applicable
* three nearby polygons, will consolidate and notify applicant

In [ ]:
new_ac = gpd.read_file('./corridors_sccp_lpp/ac_dec_15rev.geojson')

In [ ]:
# new_ac.explore()

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(4, dt.date(2022, 5, 5))

In [ ]:
rt_day.add_corridor(new_ac)

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
rt_day.segment_speed_map(corridor=True)

In [ ]:
ac_corridor_metrics = sccp_tools.sccp_average_metrics(itp_id=4, date_range=date_range, corridor=new_ac)

In [ ]:
ac_corridor_metrics

# Samtrans

In [ ]:
# itp_id = 290 # Samtrans

In [ ]:
# corridor = gpd.read_file('./corridors_sccp_lpp/peninsula.geojson') ## foster city / hayward park

# Metro (Inglewood)

In [ ]:
# itp_id = 182
# corridor = gpd.read_file('./corridors_sccp_lpp/inglewood_map.geojson')